In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdMolDescriptors, MACCSkeys
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

# Load dataset
input_csv = 'Kd_data_with_pKd.csv'  # Update with actual path
data = pd.read_csv(input_csv)

if 'canonical_smiles' not in data.columns:
    raise ValueError("The CSV file must contain a 'canonical_smiles' column.")

# Convert SMILES to RDKit Molecule objects
data['molecule'] = data['canonical_smiles'].apply(Chem.MolFromSmiles)
data = data[data['molecule'].notnull()].reset_index(drop=True)  # Remove invalid molecules

# Compute Molecular Descriptors
def calculate_descriptors(mol):
    return pd.Series({
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'TPSA': Descriptors.TPSA(mol),
        'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
        'MolMR': Descriptors.MolMR(mol),
        'BalabanJ': Descriptors.BalabanJ(mol),
        'BertzCT': Descriptors.BertzCT(mol),
        'RingCount': Descriptors.RingCount(mol),
        'HallKierAlpha': Descriptors.HallKierAlpha(mol),
        'FractionCSP3': Descriptors.FractionCSP3(mol),
        # 'EState_VSA1': rdMolDescriptors.CalcEStateIndices(mol)[0],
        # 'WienerIndex': Descriptors.WienerIndex(mol)
    })

descriptors_df = data['molecule'].apply(calculate_descriptors)

# Compute 3D Features
def calculate_3D_descriptors(mol):
    try:
        mol_3D = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol_3D, AllChem.ETKDG())
        AllChem.UFFOptimizeMolecule(mol_3D)
        return pd.Series({'MolVolume3D': rdMolDescriptors.CalcExactMolWt(mol_3D)})
    except:
        return pd.Series({'MolVolume3D': np.nan})

features_3D_df = data['molecule'].apply(calculate_3D_descriptors)

# Graph-based Features
def extract_graph_features(mol):
    G = nx.Graph(GetAdjacencyMatrix(mol))
    return {
        "degree_centrality": np.mean(list(nx.degree_centrality(G).values())),
        "closeness_centrality": np.mean(list(nx.closeness_centrality(G).values())),
        "betweenness_centrality": np.mean(list(nx.betweenness_centrality(G).values())),
        "clustering_coefficient": nx.average_clustering(G),
        "graph_density": nx.density(G)
    }

graph_features_df = data['molecule'].apply(extract_graph_features).apply(pd.Series)

# Compute Fingerprints
data['Morgan_Fingerprint'] = data['molecule'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits=1024))
data['MACCS_Fingerprint'] = data['molecule'].apply(lambda x: MACCSkeys.GenMACCSKeys(x))

# Convert Fingerprints to DataFrame
fingerprints_morgan_df = data['Morgan_Fingerprint'].apply(lambda x: pd.Series(list(map(int, x.ToBitString()))))
fingerprints_maccs_df = data['MACCS_Fingerprint'].apply(lambda x: pd.Series(list(map(int, x.ToBitString()))))

# Combine all extracted features
final_features_df = pd.concat([data, descriptors_df, features_3D_df, graph_features_df], axis=1)
final_fingerprints_df = pd.concat([fingerprints_morgan_df, fingerprints_maccs_df], axis=1)

# Save Outputs
final_features_df.to_csv('Kd_output_with_advanced_features.csv', index=False)
final_fingerprints_df.to_csv('Kd_output_with_extended_fingerprints.csv', index=False)

print("Feature extraction completed successfully.")


[02:31:14] UFFTYPER: Unrecognized charge state for atom: 12
[02:31:17] UFFTYPER: Unrecognized charge state for atom: 12
[02:31:17] UFFTYPER: Unrecognized charge state for atom: 12
[02:31:20] UFFTYPER: Unrecognized charge state for atom: 12
[02:36:35] UFFTYPER: Unrecognized charge state for atom: 23
[02:36:35] UFFTYPER: Unrecognized charge state for atom: 23
[02:40:36] UFFTYPER: Unrecognized charge state for atom: 1
[02:40:36] UFFTYPER: Unrecognized charge state for atom: 1
[02:40:42] UFFTYPER: Unrecognized charge state for atom: 1
[02:40:42] UFFTYPER: Unrecognized charge state for atom: 1
[02:40:52] UFFTYPER: Unrecognized charge state for atom: 1
[02:40:52] UFFTYPER: Unrecognized charge state for atom: 1
[05:32:21] UFFTYPER: Unrecognized charge state for atom: 1
[05:32:21] UFFTYPER: Unrecognized charge state for atom: 1
[06:35:27] UFFTYPER: Unrecognized charge state for atom: 1
[06:35:27] UFFTYPER: Unrecognized charge state for atom: 1
[06:35:55] UFFTYPER: Unrecognized charge state for

Feature extraction completed successfully.


In [10]:
import pandas as pd

# File paths
features_path = 'Kd_output_with_advanced_features.csv'
fingerprints_path = 'Kd_output_with_extended_fingerprints.csv'

# Load data
features_df = pd.read_csv(features_path)
fingerprints_df = pd.read_csv(fingerprints_path)

# Drop rows with NaN values
features_df_cleaned = features_df.dropna()
fingerprints_df_cleaned = fingerprints_df.dropna()

# Save cleaned data back to CSV
features_df_cleaned.to_csv(features_path, index=False)
fingerprints_df_cleaned.to_csv(fingerprints_path, index=False)

# Print info on removed rows
print(f"Rows removed from {features_path}: {len(features_df) - len(features_df_cleaned)}")
print(f"Rows removed from {fingerprints_path}: {len(fingerprints_df) - len(fingerprints_df_cleaned)}")


Rows removed from Kd_output_with_advanced_features.csv: 65
Rows removed from Kd_output_with_extended_fingerprints.csv: 0


In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, mean_squared_error
import joblib

# Load the dataset
features_path = 'Kd_output_with_advanced_features.csv'
fingerprints_path = 'Kd_output_with_extended_fingerprints.csv'
features_df = pd.read_csv(features_path)
fingerprints_df = pd.read_csv(fingerprints_path)

# Ensure the target column pKd is present
if 'pKd' not in features_df.columns:
    raise ValueError("The features CSV file must contain a 'pKd' column.")

# Combine features and fingerprints
combined_df = pd.concat([features_df, fingerprints_df], axis=1)

# Drop rows with NaN values
combined_df.dropna(inplace=True)

# Feature Selection
features = combined_df.drop(columns=['pKd', 'canonical_smiles', 'molecule', 'Morgan_Fingerprint', 'MACCS_Fingerprint', 'target_pref_name',
                                     'molecule_chembl_id', 'standard_value', 'standard_type', 'target_organism', 'Kd_M'])
target = combined_df['pKd']

# Scale features using RobustScaler (better for outliers)
scaler = RobustScaler()
X_scaled = scaler.fit_transform(features)
y = target.values.reshape(-1, 1)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Train-test split
train_size = int(0.8 * len(X_tensor))
test_size = len(X_tensor) - train_size
train_data, test_data = random_split(TensorDataset(X_tensor, y_tensor), [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Define Neural Network
class AdvancedNN(nn.Module):
    def __init__(self, input_dim):
        super(AdvancedNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),

            nn.Linear(128, 1)  # Output layer
        )

    def forward(self, x):
        return self.model(x)

# Model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AdvancedNN(input_dim=X_scaled.shape[1]).to(device)

# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.RAdam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Training loop
num_epochs = 200
best_loss = float("inf")
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    scheduler.step()
    
    # Validation
    model.eval()
    test_preds, test_targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            preds = model(X_batch)
            test_preds.extend(preds.cpu().numpy())
            test_targets.extend(y_batch.cpu().numpy())

    test_r2 = r2_score(test_targets, test_preds)
    test_mse = mean_squared_error(test_targets, test_preds)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Test R2: {test_r2:.4f} - Test MSE: {test_mse:.4f}")

    # Save the best model based on test MSE
    if epoch == 0 or test_mse < best_loss:
        best_loss = test_mse
        torch.save(model.state_dict(), "best_advanced_nn.pth")

# Save the scaler
joblib.dump(scaler, "scaler_advanced.joblib")

# Load best model
model.load_state_dict(torch.load("best_advanced_nn.pth"))
model.eval()

# Final evaluation
test_preds, test_targets = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch)
        test_preds.extend(preds.cpu().numpy())
        test_targets.extend(y_batch.cpu().numpy())

final_r2 = r2_score(test_targets, test_preds)
final_mse = mean_squared_error(test_targets, test_preds)

print(f"Final Model R2: {final_r2:.4f}")
print(f"Final Model MSE: {final_mse:.4f}")

# Save final results
final_results = pd.DataFrame({'Actual': test_targets.flatten(), 'Predicted': test_preds.flatten()})
final_results.to_csv("DeepLearning_Predictions_Advanced.csv", index=False)
print("Predictions saved to DeepLearning_Predictions_Advanced.csv")

Epoch 1/200 - Train Loss: 24.2319 - Test R2: -16.6709 - Test MSE: 28.1686
Epoch 2/200 - Train Loss: 21.6421 - Test R2: -14.6516 - Test MSE: 24.9498
Epoch 3/200 - Train Loss: 19.2262 - Test R2: -12.9782 - Test MSE: 22.2823
Epoch 4/200 - Train Loss: 16.8128 - Test R2: -11.8737 - Test MSE: 20.5215
Epoch 5/200 - Train Loss: 14.7956 - Test R2: -10.8350 - Test MSE: 18.8659
Epoch 6/200 - Train Loss: 12.9115 - Test R2: -9.6060 - Test MSE: 16.9067
Epoch 7/200 - Train Loss: 11.6315 - Test R2: -9.1548 - Test MSE: 16.1875
Epoch 8/200 - Train Loss: 10.8659 - Test R2: -8.8573 - Test MSE: 15.7132
Epoch 9/200 - Train Loss: 10.3802 - Test R2: -8.3448 - Test MSE: 14.8963
Epoch 10/200 - Train Loss: 10.0534 - Test R2: -8.4769 - Test MSE: 15.1068
Epoch 11/200 - Train Loss: 10.0836 - Test R2: -8.5094 - Test MSE: 15.1587
Epoch 12/200 - Train Loss: 10.0566 - Test R2: -8.4634 - Test MSE: 15.0853
Epoch 13/200 - Train Loss: 9.8141 - Test R2: -8.2790 - Test MSE: 14.7914
Epoch 14/200 - Train Loss: 9.2553 - Test R2

C:\Users\MK\AppData\Local\Temp\ipykernel_13920\2708600858.py:133: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_advanced_nn.pth"))


AttributeError: 'list' object has no attribute 'flatten'

In [3]:
combined_df.columns

Index(['target_pref_name', 'canonical_smiles', 'molecule_chembl_id',
       'standard_value', 'standard_type', 'target_organism', 'Kd_M', 'pKd',
       'molecule', 'Morgan_Fingerprint',
       ...
       '157.1', '158.1', '159.1', '160.1', '161.1', '162.1', '163.1', '164.1',
       '165.1', '166.1'],
      dtype='object', length=1220)

In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, mean_squared_error
import joblib

# Load the dataset
features_path = 'Kd_output_with_advanced_features.csv'
fingerprints_path = 'Kd_output_with_extended_fingerprints.csv'
features_df = pd.read_csv(features_path)
fingerprints_df = pd.read_csv(fingerprints_path)

# Ensure the target column pKd is present
if 'pKd' not in features_df.columns:
    raise ValueError("The features CSV file must contain a 'pKd' column.")

# Combine features and fingerprints
combined_df = pd.concat([features_df, fingerprints_df], axis=1)

# Drop rows with NaN values
combined_df.dropna(inplace=True)

# Feature Selection
features = combined_df.drop(columns=['pKd', 'canonical_smiles', 'molecule', 'Morgan_Fingerprint', 'MACCS_Fingerprint', 'target_pref_name',
                                     'molecule_chembl_id', 'standard_value', 'standard_type', 'target_organism', 'Kd_M'])
target = combined_df['pKd']

# Scale features using RobustScaler (better for outliers)
scaler = RobustScaler()
X_scaled = scaler.fit_transform(features)
y = target.values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print(f"Train R2: {train_r2:.4f}")
print(f"Train MSE: {train_mse:.4f}")
print(f"Test R2: {test_r2:.4f}")
print(f"Test MSE: {test_mse:.4f}")

# Save the model and scaler
joblib.dump(model, "rf_model.joblib")
joblib.dump(scaler, "scaler.joblib")

# Save final predictions
final_results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
final_results.to_csv("ML_Predictions.csv", index=False)
print("Predictions saved to ML_Predictions.csv")

Train R2: 0.9129
Train MSE: 0.1401
Test R2: 0.4955
Test MSE: 0.8671
Predictions saved to ML_Predictions.csv


In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, mean_squared_error
import joblib

# Load the dataset
features_path = 'Kd_output_with_advanced_features.csv'
fingerprints_path = 'Kd_output_with_extended_fingerprints.csv'
features_df = pd.read_csv(features_path)
fingerprints_df = pd.read_csv(fingerprints_path)

# Ensure the target column pKd is present
if 'pKd' not in features_df.columns:
    raise ValueError("The features CSV file must contain a 'pKd' column.")

# Combine features and fingerprints
combined_df = pd.concat([features_df, fingerprints_df], axis=1)

# Drop rows with NaN values
combined_df.dropna(inplace=True)

# Feature Selection (Remove non-numeric and unnecessary columns)
drop_cols = ['pKd', 'canonical_smiles', 'molecule', 'Morgan_Fingerprint', 
             'MACCS_Fingerprint', 'target_pref_name', 'molecule_chembl_id', 
             'standard_value', 'standard_type', 'target_organism', 'Kd_M']

features = combined_df.drop(columns=drop_cols, errors='ignore')
target = combined_df['pKd']

# Scale features using RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(features)
y = target.values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ---------------------- Overfitting Fix: Feature Selection ---------------------- #
# Train a baseline RandomForest model to get feature importance
baseline_model = RandomForestRegressor(n_estimators=100, random_state=42)
baseline_model.fit(X_train, y_train)

# Get feature importances
importances = baseline_model.feature_importances_
feature_names = features.columns
sorted_idx = np.argsort(importances)[::-1]

# Select top 20 most important features
top_n = 20  
selected_features = [feature_names[i] for i in sorted_idx[:top_n]]
X_train_selected = X_train[:, sorted_idx[:top_n]]
X_test_selected = X_test[:, sorted_idx[:top_n]]

# ---------------------- Overfitting Fix: Optimized RandomForest ---------------------- #
rf_model = RandomForestRegressor(
    n_estimators=200, max_depth=10, 
    max_features="sqrt", min_samples_split=5, 
    min_samples_leaf=2, random_state=42
)

rf_model.fit(X_train_selected, y_train)
y_pred_train = rf_model.predict(X_train_selected)
y_pred_test = rf_model.predict(X_test_selected)

train_r2 = r2_score(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print(f"RandomForest Train R2: {train_r2:.4f}, MSE: {train_mse:.4f}")
print(f"RandomForest Test R2: {test_r2:.4f}, MSE: {test_mse:.4f}")

# ---------------------- Overfitting Fix: Cross-Validation ---------------------- #
cv_r2 = cross_val_score(rf_model, X_train_selected, y_train, cv=5, scoring='r2')
print(f"Cross-validated R² scores: {cv_r2}")
print(f"Mean CV R²: {cv_r2.mean():.4f}")

# ---------------------- Alternative Models for Comparison ---------------------- #
# Gradient Boosting Model
gb_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42)
gb_model.fit(X_train_selected, y_train)
y_pred_gb = gb_model.predict(X_test_selected)
print(f"GradientBoosting Test R²: {r2_score(y_test, y_pred_gb):.4f}")

# ElasticNet (Linear Model with Regularization)
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_model.fit(X_train_selected, y_train)
y_pred_elastic = elastic_model.predict(X_test_selected)
print(f"ElasticNet Test R²: {r2_score(y_test, y_pred_elastic):.4f}")

# ---------------------- Save Best Model ---------------------- #
joblib.dump(rf_model, "kdoptimized_rf_model.joblib")
joblib.dump(scaler, "kdscaler.joblib")

# Save final predictions
final_results = pd.DataFrame({'Actual': y_test, 'RF_Predicted': y_pred_test, 
                              'GB_Predicted': y_pred_gb, 'ElasticNet_Predicted': y_pred_elastic})
final_results.to_csv("ML_Predictions.csv", index=False)
print("Predictions saved to ML_Predictions.csv")


RandomForest Train R2: 0.5723, MSE: 0.6877
RandomForest Test R2: 0.4940, MSE: 0.8696
Cross-validated R² scores: [0.44155    0.43924761 0.44402884 0.48787995 0.42698111]
Mean CV R²: 0.4479
GradientBoosting Test R²: 0.5687
ElasticNet Test R²: 0.0824
Predictions saved to ML_Predictions.csv


In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, mean_squared_error
import joblib

# Load the dataset
features_path = 'Kd_output_with_advanced_features.csv'
fingerprints_path = 'Kd_output_with_extended_fingerprints.csv'
features_df = pd.read_csv(features_path)
fingerprints_df = pd.read_csv(fingerprints_path)

# Ensure the target column pKd is present
if 'pKd' not in features_df.columns:
    raise ValueError("The features CSV file must contain a 'pKd' column.")

# Combine features and fingerprints
combined_df = pd.concat([features_df, fingerprints_df], axis=1)

# Drop rows with NaN values
combined_df.dropna(inplace=True)

# Feature Selection (Remove non-numeric and unnecessary columns)
drop_cols = ['pKd', 'canonical_smiles', 'molecule', 'Morgan_Fingerprint', 
             'MACCS_Fingerprint', 'target_pref_name', 'molecule_chembl_id', 
             'standard_value', 'standard_type', 'target_organism', 'Kd_M']

features = combined_df.drop(columns=drop_cols, errors='ignore')
target = combined_df['pKd']

# Scale features using RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(features)
y = target.values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ---------------------- Overfitting Fix: Feature Selection ---------------------- #
# Train a baseline RandomForest model to get feature importance
baseline_model = RandomForestRegressor(n_estimators=100, random_state=42)
baseline_model.fit(X_train, y_train)

# Get feature importances
importances = baseline_model.feature_importances_
feature_names = features.columns
sorted_idx = np.argsort(importances)[::-1]

# Select top 20 most important features
top_n = 20  
selected_features = [feature_names[i] for i in sorted_idx[:top_n]]
X_train_selected = X_train[:, sorted_idx[:top_n]]
X_test_selected = X_test[:, sorted_idx[:top_n]]

# ---------------------- Overfitting Fix: Optimized RandomForest ---------------------- #
rf_model = RandomForestRegressor(
    n_estimators=200, max_depth=10, 
    max_features="sqrt", min_samples_split=5, 
    min_samples_leaf=2, random_state=42
)

rf_model.fit(X_train_selected, y_train)
y_pred_train = rf_model.predict(X_train_selected)
y_pred_test = rf_model.predict(X_test_selected)

train_r2 = r2_score(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print(f"RandomForest Train R2: {train_r2:.4f}, MSE: {train_mse:.4f}")
print(f"RandomForest Test R2: {test_r2:.4f}, MSE: {test_mse:.4f}")

# ---------------------- Overfitting Fix: Cross-Validation ---------------------- #
cv_r2 = cross_val_score(rf_model, X_train_selected, y_train, cv=5, scoring='r2')
print(f"Cross-validated R² scores: {cv_r2}")
print(f"Mean CV R²: {cv_r2.mean():.4f}")

# ---------------------- Alternative Models for Comparison ---------------------- #
# Gradient Boosting Model
gb_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42)
gb_model.fit(X_train_selected, y_train)
y_pred_gb = gb_model.predict(X_test_selected)
print(f"GradientBoosting Test R²: {r2_score(y_test, y_pred_gb):.4f}")

# ElasticNet (Linear Model with Regularization)
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_model.fit(X_train_selected, y_train)
y_pred_elastic = elastic_model.predict(X_test_selected)
print(f"ElasticNet Test R²: {r2_score(y_test, y_pred_elastic):.4f}")

# ---------------------- Save Best Model ---------------------- #
joblib.dump(rf_model, "kdoptimized_rf_model.joblib")
joblib.dump(scaler, "kdscaler.joblib")

# Save final predictions
final_results = pd.DataFrame({'Actual': y_test, 'RF_Predicted': y_pred_test, 
                              'GB_Predicted': y_pred_gb, 'ElasticNet_Predicted': y_pred_elastic})
final_results.to_csv("ML_Predictions.csv", index=False)
print("Predictions saved to ML_Predictions.csv")

RandomForest Train R2: 0.5723, MSE: 0.6877
RandomForest Test R2: 0.4940, MSE: 0.8696
Cross-validated R² scores: [0.44155    0.43924761 0.44402884 0.48787995 0.42698111]
Mean CV R²: 0.4479
GradientBoosting Test R²: 0.5687
ElasticNet Test R²: 0.0824
Predictions saved to ML_Predictions.csv


In [17]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, mean_squared_error
import joblib

# Load the dataset
features_path = 'Kd_output_with_advanced_features.csv'
fingerprints_path = 'Kd_output_with_extended_fingerprints.csv'
features_df = pd.read_csv(features_path)
fingerprints_df = pd.read_csv(fingerprints_path)

# Ensure the target column pKd is present
if 'pKd' not in features_df.columns:
    raise ValueError("The features CSV file must contain a 'pKd' column.")

# Combine features and fingerprints
combined_df = pd.concat([features_df, fingerprints_df], axis=1)

# Drop rows with NaN values
combined_df.dropna(inplace=True)

# Feature Selection (Remove non-numeric and unnecessary columns)
drop_cols = ['pKd', 'canonical_smiles', 'molecule', 'Morgan_Fingerprint', 
             'MACCS_Fingerprint', 'target_pref_name', 'molecule_chembl_id', 
             'standard_value', 'standard_type', 'target_organism', 'Kd_M']

features = combined_df.drop(columns=drop_cols, errors='ignore')
target = combined_df['pKd']

# Scale features using RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(features)
y = target.values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ---------------------- Overfitting Fix: Feature Selection ---------------------- #
# Train a baseline RandomForest model to get feature importance
baseline_model = RandomForestRegressor(n_estimators=100, random_state=42)
baseline_model.fit(X_train, y_train)

# Get feature importances
importances = baseline_model.feature_importances_
feature_names = features.columns
sorted_idx = np.argsort(importances)[::-1]

# Select top 20 most important features
top_n = 20  
selected_features = [feature_names[i] for i in sorted_idx[:top_n]]
selected_indices = sorted_idx[:top_n]
X_train_selected = X_train[:, sorted_idx[:top_n]]
X_test_selected = X_test[:, sorted_idx[:top_n]]

# Save the selected feature indices
np.save("selected_feature_indices.npy", selected_indices)

# ---------------------- Overfitting Fix: Optimized RandomForest ---------------------- #
rf_model = RandomForestRegressor(
    n_estimators=200, max_depth=10, 
    max_features="sqrt", min_samples_split=5, 
    min_samples_leaf=2, random_state=42
)

rf_model.fit(X_train_selected, y_train)
y_pred_train = rf_model.predict(X_train_selected)
y_pred_test = rf_model.predict(X_test_selected)

train_r2 = r2_score(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print(f"RandomForest Train R2: {train_r2:.4f}, MSE: {train_mse:.4f}")
print(f"RandomForest Test R2: {test_r2:.4f}, MSE: {test_mse:.4f}")

# ---------------------- Overfitting Fix: Cross-Validation ---------------------- #
cv_r2 = cross_val_score(rf_model, X_train_selected, y_train, cv=5, scoring='r2')
print(f"Cross-validated R² scores: {cv_r2}")
print(f"Mean CV R²: {cv_r2.mean():.4f}")

# ---------------------- Alternative Models for Comparison ---------------------- #
# Gradient Boosting Model
gb_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42)
gb_model.fit(X_train_selected, y_train)
y_pred_gb = gb_model.predict(X_test_selected)
print(f"GradientBoosting Test R²: {r2_score(y_test, y_pred_gb):.4f}")

# ElasticNet (Linear Model with Regularization)
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_model.fit(X_train_selected, y_train)
y_pred_elastic = elastic_model.predict(X_test_selected)
print(f"ElasticNet Test R²: {r2_score(y_test, y_pred_elastic):.4f}")

# ---------------------- Save Best Model ---------------------- #
joblib.dump(rf_model, "kdoptimized_rf_model.joblib")
joblib.dump(scaler, "kdscaler.joblib")

# Save final predictions
final_results = pd.DataFrame({'Actual': y_test, 'RF_Predicted': y_pred_test, 
                              'GB_Predicted': y_pred_gb, 'ElasticNet_Predicted': y_pred_elastic})
final_results.to_csv("ML_Predictions.csv", index=False)
print("Predictions saved to ML_Predictions.csv")

RandomForest Train R2: 0.5723, MSE: 0.6877
RandomForest Test R2: 0.4940, MSE: 0.8696
Cross-validated R² scores: [0.44155    0.43924761 0.44402884 0.48787995 0.42698111]
Mean CV R²: 0.4479
GradientBoosting Test R²: 0.5687
ElasticNet Test R²: 0.0824
Predictions saved to ML_Predictions.csv
